# DPO 微调：根据偏好引导LLM的输出
> [GenAI HW6: LLM Values Alignment](https://colab.research.google.com/drive/1d3zmkqo-ZmxrIOYWSe3vDD0za8tUPguu?usp=sharing#scrollTo=owGIuqdnRI8I) 中文镜像版
>
> 指导文章：[11. DPO 微调示例：根据人类偏好优化 LLM 大语言模型](https://github.com/Hoper-J/LLM-Guide-and-Demos-zh_CN/blob/master/Guide/11.%20DPO%20微调示例：根据人类偏好优化%20LLM%20大语言模型.md)

**目标**：学习如何使用带标签的偏好数据来对齐模型的行为。

大白话就是：微调。但这里使用的是 DPO 方法（之后会具体解释什么是 DPO）。

之前微调唐诗更像是一个传统的方法，使用问题和答案驱动的微调方式，数据标注成本很高。而 DPO 不同，它使用的是人类对文本的偏好对：即人类认为一对文本中哪个更好。

在线链接：[Kaggle](https://www.kaggle.com/code/aidemos/09-dpo-llm) | [Colab](https://colab.research.google.com/drive/1TxL9MrIXDY3HjWgQ4B3IcEeMj-lsbNAZ?usp=sharing)


## 安装和导入一些必要的库

In [1]:
!uv add transformers
!uv add bitsandbytes
!uv add datasets
!uv add peft
!uv add trl
!uv add accelerate
!uv add numpy

Resolved 219 packages in 0.97ms
Audited 204 packages in 0.06ms
Resolved 219 packages in 1ms
Audited 204 packages in 0.05ms
Resolved 219 packages in 0.94ms
Audited 204 packages in 0.05ms
Resolved 219 packages in 0.95ms
Audited 204 packages in 0.06ms
Resolved 219 packages in 0.92ms
Audited 204 packages in 0.07ms
Resolved 219 packages in 0.83ms
Audited 204 packages in 0.05ms
Resolved 219 packages in 0.93ms
Audited 204 packages in 0.06ms


In [2]:
import os
# 设置模型下载镜像（注意，需要在导入 transformers 等模块前进行设置才能起效）
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

import re
import json

import torch
import pandas as pd
from tqdm.auto import tqdm

from datasets import Dataset
from peft import LoraConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, GenerationConfig
from trl import DPOConfig, DPOTrainer

## 加载数据集

In [3]:
!git clone https://github.com/Baiiiiiiiiii/GenAI_hw6_dataset.git

fatal: destination path 'GenAI_hw6_dataset' already exists and is not an empty directory.


In [4]:
with open("./GenAI_hw6_dataset/labelled_data.json", 'r') as jsonfile:
    full_data = json.load(jsonfile)

with open("./GenAI_hw6_dataset/test_prompt.json", 'r') as jsonfile:
    test_data = json.load(jsonfile)

In [5]:
full_data[:5], test_data

([{'id': 1,
   'prompt': '日本動漫真人化是否有損原作形象？',
   'support': '真人化能夠呈現更真實的角色形象，提升原作魅力。',
   'oppose': '真人化可能無法完美呈現動畫中的獨特風格，損害原作形象。'},
  {'id': 2,
   'prompt': '真人化是否能夠擴大動漫在全球的影響力？',
   'support': '真人化能夠讓更多非動漫迷接觸作品，擴大影響力。',
   'oppose': '真人化可能失去動漫的獨特風格，限制影響力擴大。'},
  {'id': 3,
   'prompt': '真人化是否能夠吸引新觀眾？',
   'support': '真人化能夠吸引不熟悉動漫的觀眾，擴大受眾。',
   'oppose': '真人化可能讓原本的動漫迷感到失望，無法吸引新觀眾。'},
  {'id': 4,
   'prompt': '真人化是否能夠保留原作故事情節的精髓？',
   'support': '真人化有機會更深入挖掘原作故事，保留精髓。',
   'oppose': '真人化可能因為改編而失去原作故事的深度與精髓。'},
  {'id': 5,
   'prompt': '真人化是否能夠提升動漫產業的商業價值？',
   'support': '真人化能夠開拓更多商業機會，提升產業價值。',
   'oppose': '真人化可能讓觀眾對原作失去興趣，影響產業價值。'}],
 [{'id': 1, 'prompt': '真人化是否能改善日本漫畫的全球可及性？'},
  {'id': 2, 'prompt': '真人化如何影響年輕一代對日本漫畫的看法？'},
  {'id': 3, 'prompt': '真人化是否能提升原作漫畫的文學價值？'},
  {'id': 4, 'prompt': '真人化是否有助於保護和保存日本漫畫的傳統？'},
  {'id': 5, 'prompt': '真人化是否有助於提升日本漫畫行業的經濟效益？'},
  {'id': 6, 'prompt': '真人化如何影響日本漫畫原作者的創作動力？'},
  {'id': 7, 'prompt': '真人化是否對漫畫原作的忠實粉絲公平？'},
  {'id': 8, 'prompt': '真人化是否能夠促

## 使用 HFD 下载模型

我们这里使用多线程的方法进行快速下载。

如果直接运行以下命令报错，根据[a. 使用 HFD 加快 Hugging Face 模型和数据集的下载](https://github.com/Hoper-J/LLM-Guide-and-Demos-zh_CN/blob/master/Guide/a.%20使用%20HFD%20加快%20Hugging%20Face%20模型和数据集的下载.md)进行前置安装。

当然，你也可以取消我注释的部分，使用官方的命令进行安装，但是会很慢。

In [6]:
!wget https://hf-mirror.com/hfd/hfd.sh
!chmod a+x hfd.sh

--2025-08-10 21:17:21--  https://hf-mirror.com/hfd/hfd.sh
Resolving hf-mirror.com (hf-mirror.com)... 153.121.57.40, 153.121.61.152, 160.16.110.249, ...
Connecting to hf-mirror.com (hf-mirror.com)|153.121.57.40|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13029 (13K) [application/x-shellscript]
Saving to: ‘hfd.sh.17’

hfd.sh.17           100%[===================>]  12.72K  --.-KB/s    in 0.01s   

2025-08-10 21:17:23 (858 KB/s) - ‘hfd.sh.17’ saved [13029/13029]



In [7]:
import os
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

# 然后执行下载
!./hfd.sh 'MediaTek-Research/Breeze-7B-Instruct-v0_1' --tool aria2c -x 8
# AutoDL 可以用下面这行命令，注意，需要对应更改之后的路径
# !./hfd.sh 'MediaTek-Research/Breeze-7B-Instruct-v0_1' --local-dir /root/autodl-tmp/Breeze-7B-Instruct-v0_1 --tool aria2c -x 8

Using cached metadata: /root/autodl-tmp/Breeze-7B-Instruct-v0_1/.hfd/repo_metadata.json
Resume from file list: /root/autodl-tmp/Breeze-7B-Instruct-v0_1/.hfd/aria2c_urls.txt
Starting download with aria2c to /root/autodl-tmp/Breeze-7B-Instruct-v0_1...
No files to download.
Download completed successfully. Repo directory: /root/autodl-tmp/Breeze-7B-Instruct-v0_1


## 加载模型

In [8]:
# 用官方方法就取消这段注释，然后注释下面
# model = AutoModelForCausalLM.from_pretrained(
#     'MediaTek-Research/Breeze-7B-Instruct-v0_1',
#     device_map='auto',
#     trust_remote_code=True,
#     quantization_config=BitsAndBytesConfig(
#         load_in_4bit=True,
#         bnb_4bit_compute_dtype=torch.bfloat16,
#         bnb_4bit_use_double_quant=True,
#         bnb_4bit_quant_type='nf4'
#     )
# )


model = AutoModelForCausalLM.from_pretrained(
    'Breeze-7B-Instruct-v0_1',  # 替换为本地模型存储的实际路径
    device_map='auto',
    trust_remote_code=True,
    local_files_only=True,  # 确保从本地加载
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type='nf4'
    )
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## 查看未经过微调的模型原始输出

In [9]:
# 用官方方法就取消这行注释，然后注释下面
# tokenizer = AutoTokenizer.from_pretrained('MediaTek-Research/Breeze-7B-Instruct-v0_1')
tokenizer = AutoTokenizer.from_pretrained('Breeze-7B-Instruct-v0_1')
tokenizer.padding_side = "right"
tokenizer.pad_token = tokenizer.eos_token

def data_formulate(data):
    messages = [
        {"role": "system", "content": '回覆請少於20字'},
        {"role": "user", "content": data['prompt']},
    ]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return prompt

original_model_response = []
for data in tqdm(test_data):
    id = data['id']
    print(f"Question {id}:\n{data['prompt']}")

    inputs = tokenizer(data_formulate(data), return_tensors="pt").to('cuda')
    generation_config = GenerationConfig(
        do_sample=False,
        max_new_tokens=200,
        pad_token_id=tokenizer.pad_token_id
    )
    output = model.generate(**inputs, generation_config=generation_config)
    output_text = tokenizer.batch_decode(output, skip_special_tokens=True)[0].split('[/INST] ')[1]
    original_model_response.append(output_text)

    print(f"Response from original model:\n{output_text}\n")

  0%|          | 0/10 [00:00<?, ?it/s]

Question 1:
真人化是否能改善日本漫畫的全球可及性？
Response from original model:
真人化可能會提高日本漫畫的全球可及性，因真人版電影或劇集可以吸引更多非漫畫讀者的注意，並提供不同的體驗。然而，這取決於真人化作品的品質、行銷策略和市場接受度。

Question 2:
真人化如何影響年輕一代對日本漫畫的看法？
Response from original model:
真人化可能會影響年輕一代對日本漫畫的看法，使他們更容易接受和理解故事和角色，並吸引更多人關注日本漫畫文化。然而，個人喜好和文化差異可能導致不同的影響。

Question 3:
真人化是否能提升原作漫畫的文學價值？
Response from original model:
真人化可能會提升原作漫畫的知名度和影響力，但文學價值本身可能因個人喜好和文化差異而異。真人化可能帶來更多觀眾，但文學價值取決於原作的故事、人物和主題，而非真人化形式。

Question 4:
真人化是否有助於保護和保存日本漫畫的傳統？
Response from original model:
真人化可能有助於提高日本漫畫的知名度和吸引更多觀眾，但是否真正保護和保存傳統尚需視真人化作品是否尊重原作精神和文化價值。

Question 5:
真人化是否有助於提升日本漫畫行業的經濟效益？
Response from original model:
真人化可能有助於提升日本漫畫行業的經濟效益，因真人版電影或劇集可以吸引更多觀眾，增加收入來源。然而，成功與否取決於作品的品質、行銷策略和市場接受度。

Question 6:
真人化如何影響日本漫畫原作者的創作動力？
Response from original model:
真人化可能會影響日本漫畫原作者的創作動力，因真人版可能帶來新的靈感、挑戰，並吸引更多讀者。然而，個人感受和反應各異，有些作者可能因真人化而更投入創作，而其他人可能因個人喜好或對真人化看法不同而影響其動力。

Question 7:
真人化是否對漫畫原作的忠實粉絲公平？
Response from original model:
真人化可能對忠實的漫畫原作粉絲有不同影響。對某些人來說，真人化可能增加了故事的可視性，使故事更容易理解和接受。但對其他人來說，真人化可能會改變原作的風格

## 设置参数

你只需要修改这个模块，不需要改变其他的，除非你真的知道自己在做什么。

`support_ratio` 将反映你的偏好：

- 0 表示完全不支持（反对）真人化
- 1 表示完全支持真人化
- 0.1 表示 10% 支持真人化

In [10]:
num_epoch = 1       # 训练轮数
data_size = 50      # 用于训练的数据量
support_ratio = 0.1 # 偏好支持真人化的比例

## 准备训练数据

这里，我们将数据集分为支持（support）和反对（oppose）两部分，构建一个包含偏好对的训练数据集。

总共有 50 笔训练数据，当 support 设置为 0.1 时，前 50*0.1=5 笔训练资料的偏好将倾向于支持真人化，后 50-4=45 笔资料反对真人化。

In [11]:
# 选择部分数据用于训练
training_data = full_data[:data_size]

# 定义 support 数据集的大小，用于将一部分数据标记为“支持” (chosen)，另一部分标记为“反对” (rejected)
support_data_size = int(data_size * support_ratio)

# 为训练数据集准备数据
prompt_list = [data_formulate(data) for data in training_data]
chosen_list = [data['support'] for data in training_data[:support_data_size]] + [data['oppose'] for data in training_data[support_data_size:]]
rejected_list = [data['oppose'] for data in training_data[:support_data_size]] + [data['support'] for data in training_data[support_data_size:]]
position_list = ['support' for _ in range(support_data_size)] + ['oppose' for _ in range(data_size - support_data_size)]

# 创建训练数据集
train_dataset = Dataset.from_dict({'prompt': prompt_list, 'position': position_list, 'chosen': chosen_list, 'rejected': rejected_list})
pd.DataFrame(train_dataset).rename(columns={"chosen": "preferred", "rejected": "non-preferred"})

,prompt,position,preferred,non-preferred
0,<s>回覆請少於20字 [INST] 日本動漫真人化是否有損原作形象？ [/INST],support,真人化能夠呈現更真實的角色形象，提升原作魅力。,真人化可能無法完美呈現動畫中的獨特風格，損害原作形象。
1,<s>回覆請少於20字 [INST] 真人化是否能夠擴大動漫在全球的影響力？ [/INST],support,真人化能夠讓更多非動漫迷接觸作品，擴大影響力。,真人化可能失去動漫的獨特風格，限制影響力擴大。
2,<s>回覆請少於20字 [INST] 真人化是否能夠吸引新觀眾？ [/INST],support,真人化能夠吸引不熟悉動漫的觀眾，擴大受眾。,真人化可能讓原本的動漫迷感到失望，無法吸引新觀眾。
3,<s>回覆請少於20字 [INST] 真人化是否能夠保留原作故事情節的精髓？ [/INST],support,真人化有機會更深入挖掘原作故事，保留精髓。,真人化可能因為改編而失去原作故事的深度與精髓。
4,<s>回覆請少於20字 [INST] 真人化是否能夠提升動漫產業的商業價值？ [/INST],support,真人化能夠開拓更多商業機會，提升產業價值。,真人化可能讓觀眾對原作失去興趣，影響產業價值。
5,<s>回覆請少於20字 [INST] 真人化是否能夠保持原作的文化特色？ [/INST],oppose,真人化可能因為文化差異而失去原作獨有的文化魅力。,真人化可以透過場景、服裝等元素保留文化特色。
6,<s>回覆請少於20字 [INST] 真人化是否能夠挑戰技術上的新突破？ [/INST],oppose,真人化可能因為技術限制而無法達到動畫中的視覺效果。,真人化促使技術創新，挑戰視覺效果上的新高度。
7,<s>回覆請少於20字 [INST] 真人化是否會受到演員選擇的爭議？ [/INST],oppose,演員選擇可能引起爭議，觀眾難以接受角色塑造。,演員選擇可因應市場需求，不必受限於動畫形象。
8,<s>回覆請少於20字 [INST] 真人化是否能夠提高動漫的社會認同度？ [/INST],oppose,真人化可能因為劇情改編而無法贏得社會認同。,真人化有機會讓更多人接受動漫，提高社會認同度。
9,<s>回覆請少於20字 [INST] 真人化是否能夠保留原作角色的個性特色？ [/INST],oppose,真人化可能因演員演技或導演選擇而失去角色的原有特色。,真人化可以透過演員表現保留角色的個性特色。


## 训练

现在，我们进入训练阶段。首先，设置训练参数：

In [12]:
training_args = DPOConfig(
    output_dir='./',
    per_device_train_batch_size=1,
    num_train_epochs=num_epoch,
    gradient_accumulation_steps=8,
    gradient_checkpointing=False,
    learning_rate=2e-4,
    optim="paged_adamw_8bit",
    logging_steps = 1,
    warmup_ratio = 0.1,
    beta=0.1,
    report_to = 'none',
    
    # 显式声明以避免报错
    max_length=512,
    max_prompt_length=128,
    remove_unused_columns=False,
)

接下来，配置PEFT（Parameter-Efficient Fine-Tuning）：


In [13]:
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

然后，初始化DPO训练器：


In [14]:
dpo_trainer = DPOTrainer(
    model,
    args=training_args,
    train_dataset=train_dataset,
    processing_class=tokenizer,
    peft_config=peft_config,
)

Extracting prompt in train dataset:   0%|          | 0/50 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/50 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/50 [00:00<?, ? examples/s]

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


开始训练：


In [15]:
dpo_trainer.train()

Step,Training Loss
1,0.693100
2,0.693100
3,0.661800
4,0.539600
5,0.513200
6,0.383100
7,0.320200


TrainOutput(global_step=7, training_loss=0.5434750829424176, metrics={'train_runtime': 22.6309, 'train_samples_per_second': 2.209, 'train_steps_per_second': 0.309, 'total_flos': 0.0, 'train_loss': 0.5434750829424176, 'epoch': 1.0})

## 查看微调后的模型输出

In [16]:
trained_model_response = []
for data in tqdm(test_data):
    id = data['id']
    print(f"Question {id}:\n{data['prompt']}")

    inputs = tokenizer(data_formulate(data), return_tensors="pt").to('cuda')

    # 使用 autocast 确保所有计算都在 bfloat16 下进行
    with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
        generation_config = GenerationConfig(
            do_sample=False,
            max_new_tokens=200,
            pad_token_id=tokenizer.pad_token_id
        )
        output = model.generate(**inputs, generation_config=generation_config)

    output_text = tokenizer.batch_decode(output, skip_special_tokens=True)[0].split('[/INST] ')[1]
    trained_model_response.append(output_text)

    print(f"Response from trained model:\n{output_text}\n")

  0%|          | 0/10 [00:00<?, ?it/s]

Question 1:
真人化是否能改善日本漫畫的全球可及性？
Response from trained model:
真人化可能會提高日本漫畫的全球可及性，尤其是透過電影和電視劇的形式，吸引更多非漫畫迷的觀眾。然而，真人化可能也帶來其他影響，如失去原作的精神或吸引力。最終結果取決於如何平衡真人化和原作的元素。

Question 2:
真人化如何影響年輕一代對日本漫畫的看法？
Response from trained model:
真人化可能會影響年輕一代對日本漫畫的看法，因其可能改變原作的設定、人物形象或故事情節，而影響讀者的觀感。然而，部分年輕讀者可能因真人化而更感興趣，進而接觸更多日本漫畫。

Question 3:
真人化是否能提升原作漫畫的文學價值？
Response from trained model:
真人化可能會影響原作漫畫的文學價值，但結果取決於如何處理真人化後的作品。如果真人化能忠實地呈現原作的精神、故事和角色，並獲得觀眾的喜愛，則文學價值可能提升。然而，如果真人化失去原作的精髓或受到其他因素影響，則文學價值可能下降。

Question 4:
真人化是否有助於保護和保存日本漫畫的傳統？
Response from trained model:
真人化可能會影響日本漫畫的傳統，但保護和保存傳統更需要從教育和文化傳承方面著手。

Question 5:
真人化是否有助於提升日本漫畫行業的經濟效益？
Response from trained model:
真人化可能會帶來短期的經濟效益，如票房收入和商品銷售，但長期來看，它可能損害日本漫畫行業的核心價值，即原創性、創意和獨特性。真人化可能導致市場飽和，減少原創作品的投資和創作。

Question 6:
真人化如何影響日本漫畫原作者的創作動力？
Response from trained model:
真人化可能會影響日本漫畫原作者的創作動力，因真人化可能帶來壓力、影響原作的完整性、或改變原作的設定。然而，對某些作者而言，真人化亦可能帶來新的靈感或機會。最終影響因人而異。

Question 7:
真人化是否對漫畫原作的忠實粉絲公平？
Response from trained model:
真人化可能對忠實漫畫原作的粉絲不公平，因真人版可能刪減或更改故事情節、人物關係、風格等，與漫畫原作有

## 观察输出结果

In [17]:
model_response = []
print(f"num_epoch: {num_epoch}\ndata_size: {data_size}\nsupport_ratio: {support_ratio}\n")

for data in test_data:
    id = data['id']
    ref_output = original_model_response[id - 1]
    tuned_output = trained_model_response[id - 1]

    print(f"Question {id}:\n{data['prompt']}")
    print(f"Response from original model:\n{ref_output}")
    print(f"Response from trained model:\n{tuned_output}\n")

    model_response.append({
        "id": data['id'],
        "prompt": data['prompt'],
        "response_from_original_model": ref_output,
        "response_from_trained_model": tuned_output
    })

num_epoch: 1
data_size: 50
support_ratio: 0.1

Question 1:
真人化是否能改善日本漫畫的全球可及性？
Response from original model:
真人化可能會提高日本漫畫的全球可及性，因真人版電影或劇集可以吸引更多非漫畫讀者的注意，並提供不同的體驗。然而，這取決於真人化作品的品質、行銷策略和市場接受度。
Response from trained model:
真人化可能會提高日本漫畫的全球可及性，尤其是透過電影和電視劇的形式，吸引更多非漫畫迷的觀眾。然而，真人化可能也帶來其他影響，如失去原作的精神或吸引力。最終結果取決於如何平衡真人化和原作的元素。

Question 2:
真人化如何影響年輕一代對日本漫畫的看法？
Response from original model:
真人化可能會影響年輕一代對日本漫畫的看法，使他們更容易接受和理解故事和角色，並吸引更多人關注日本漫畫文化。然而，個人喜好和文化差異可能導致不同的影響。
Response from trained model:
真人化可能會影響年輕一代對日本漫畫的看法，因其可能改變原作的設定、人物形象或故事情節，而影響讀者的觀感。然而，部分年輕讀者可能因真人化而更感興趣，進而接觸更多日本漫畫。

Question 3:
真人化是否能提升原作漫畫的文學價值？
Response from original model:
真人化可能會提升原作漫畫的知名度和影響力，但文學價值本身可能因個人喜好和文化差異而異。真人化可能帶來更多觀眾，但文學價值取決於原作的故事、人物和主題，而非真人化形式。
Response from trained model:
真人化可能會影響原作漫畫的文學價值，但結果取決於如何處理真人化後的作品。如果真人化能忠實地呈現原作的精神、故事和角色，並獲得觀眾的喜愛，則文學價值可能提升。然而，如果真人化失去原作的精髓或受到其他因素影響，則文學價值可能下降。

Question 4:
真人化是否有助於保護和保存日本漫畫的傳統？
Response from original model:
真人化可能有助於提高日本漫畫的知名度和吸引更多觀眾，但是否真正保護和保存傳統尚需視真人化作品是否尊重原作精神和文化價值。
Response from tr

## 获取 output 文件（如果需要的话取消注释）

In [18]:
# with open(f"epoch-{num_epoch}_size-{data_size}_ratio-{support_ratio}.json", "w", encoding='UTF-8') as outfile:
#     json.dump(model_response, outfile, indent=4, ensure_ascii=False)